# Práctica-08: Base de datos de satélites UCS

**Objetivo:** Analizar detalles sobre los satélites que actualmente orbitan la Tierra, incluido su país de origen, propósito y otros detalles operativos.

La base de datos se actualiza tres veces al año. La base de datos contiene 28 tipos de datos para cada satélite, incluida información técnica sobre cada satélite (_masa, potencia, fecha de lanzamiento, vida útil esperada_) y su órbita (_apogeo, perigeo, inclinación y período_), así como información sobre lo que hace el satélite.

- Vamos a presentar números básicos sobre la cadena de valor de los servicios satelitales (como datos sobre lanzadores, fabricantes de satélites, operadores de satélites) y la industria espacial en general.

¿Alguna vez te has preguntado cuántos satélites hay encima de nosotros?

**Primer paso: cargar la base de datos**

Estableceremos el índice del marco de datos a la fecha de lanzamiento de cada satélite, contenido en la columna "Fecha de lanzamiento" (Date of Launch)

In [11]:
import numpy as np  # útil para muchos cálculos científicos en Python
import pandas as pd # biblioteca de estructura de datos primaria
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

sns.color_palette("colorblind", n_colors=8, desat=.5)
plt.style.use('tableau-colorblind10')

# Esta función personaliza un objeto de barra de gráfico de barras para agregar el número de observaciones.
def set_bar_label(ax_ref, minval=1, orient='h'):
  for p in ax_ref.patches:
    if orient=='h':    
      if p.get_width() >=minval:
        ax_ref.annotate('{}'.format(p.get_width()),xy=(p.get_x()+p.get_width()*1.01,p.get_y()*1.05),fontsize=13)
    else:
      if p.get_height() >=minval:
          ax_ref.annotate('{}'.format(p.get_height()),xy=(p.get_x()+0.4*p.get_width(), p.get_y() + p.get_height()*1.01),fontsize=13)

In [12]:
df_sat=pd.read_excel('UCS-Satellite-Database-1-1-2022.xls',index_col='Date of Launch',parse_dates=True,thousands=',')
df_sat.index.name='Date of Launch'
print('Datos cargados')

El primer paso cuando se trabaja con datos es _explorarlos_. El objetivo de este primer paso es identificar tipos de datos, datos faltantes, problemas de formato, etc., que puedan existir en los datos.

Los métodos **.info()** y **.head()** son útiles para determinar los tipos de datos y los datos faltantes. Observemos los nombres de las columnas, el conteo de datos y el tipo de datos.

In [ ]:
df_sat.info()

**Segundo paso: limpiar el conjunto de datos**

Algunos de los campos de datos más interesantes incluyen la fecha de lanzamiento del satélite, el fabricante y operador del satélite, el propósito del satélite, el tipo de órbita, incluidos algunos parámetros orbitales, y la capacidad de masa y potencia del satélite.

Pero también, no es un conjunto de datos perfecto. Vemos los siguientes problemas:

- Faltan datos en algunas de las columnas más interesantes, por ejemplo, carga útil, Masa, Potencia.
- Columnas vacías.

Para tener una idea de cómo se ven los datos, exploremos las primeras y la última fila de datos, usando los métodos **.head()** y **.last()**

In [ ]:
df_sat.head()

In [ ]:
df_sat.tail(2)

**Tipos de datos y datos faltantes**

Como se mencionó anteriormente, la base de datos contiene algunas columnas con comentarios y puntos de datos faltantes. También contiene filas completamente vacías, como se observó en la salida del método **.tail()**. 

Para nuestro trabajo exploratorio, se descartan las últimas filas innecesarias, que no contienen datos en la columna "_Nombre del satélite, nombres alternativos_", todas las columnas que no contengan más de 5 puntos de datos válidos (es decir, al menos 5 filas con datos válidos en la columna, usando el método **.dropna()**), y finalmente se convertirá a numéricas las columnas que deben contener datos numéricos.

Luego, después de eliminar las filas problemáticas a las que les faltan las fechas de lanzamiento, ordenaremos el índice (fecha de lanzamiento) cronológicamente. 

In [ ]:
#quitar filas con NaN,
df_sat = df_sat[ df_sat['Name of Satellite, Alternate Names'].notna() ]

#soltar columnas con menos de 5 elementos válidos
df_sat.dropna( axis='columns', thresh=5, inplace=True)

#corregir los espacios adicionales en las columnas de categoría
df_sat['Users']=df_sat['Users'].str.strip()

#asegúrese de que las columnas numéricas sean del tipo correcto
df_sat[['Dry Mass (kg.)', 'Launch Mass (kg.)', 'Eccentricity', 'Inclination (degrees)','Period (minutes)', 'Power (watts)']]=\
    df_sat[['Dry Mass (kg.)', 'Launch Mass (kg.)', 'Eccentricity', 'Inclination (degrees)','Period (minutes)', 'Power (watts)']]\
                                                                                            .apply(pd.to_numeric,errors='coerce')
#ordenar el índice.
df_sat.sort_index(axis=0, inplace=True, ascending=True)
df_sat.info()

Además de esto, ahora es importante explorar las columnas numéricas para asegurarse de que no haya ceros donde puedan ser problemáticos. Por ejemplo, un cero en las columnas _Masa seca, Masa de lanzamiento, Potencia_ no es un punto de datos correcto. Después de inspeccionar, el satélite NSS-6 parece tener un cero en la columna Potencia (vatios).

In [ ]:
## identificar CERO en las columnas de masa o potencia
isZeroPower_idx = df_sat['Power (watts)'] == 0
isZeroDryMass_idx = df_sat['Dry Mass (kg.)'] == 0
isZeroLaunchMass_idx = df_sat['Launch Mass (kg.)'] == 0

print('Número de entradas en las columnas Potencia, Masa de lanzamiento y Masa seca establecidas en cero')
print('-------------------------------------------------------------------------')
print(' Número de puntos de datos que faltan en la columna Potencia: {}'.format(isZeroPower_idx.sum()))
print(df_sat[isZeroPower_idx]['Current Official Name of Satellite'].to_string())
print('-------------------------------------------------------------------------')
print(' Número de puntos de datos faltantes en columna de Masa seca: {}'.format(isZeroDryMass_idx.sum()))
print(' Número de puntos de datos que faltan en la columna Masa de lanzamiento: {}'.format(isZeroLaunchMass_idx.sum()))

Según la información disponible en una versión anterior de la base de datos (UCS-Satellite-Database-8-1-2020.xls), la plataforma del satélite entrega 10 KW de potencia.

In [ ]:
# Establezca el valor de POTENCIA (vatios) para NSS-6 en 10000
df_sat.loc[isZeroPower_idx,'Power (watts)'] = 10000
print(df_sat[isZeroPower_idx]['Power (watts)'].to_string())

Ahora el marco de datos está más limpio y podemos comenzar nuestro trabajo exploratorio.

Las **estaciones terrestres de satélites** son instalaciones diseñadas con el fin de proporcionar una comunicación directa y en tiempo real con los satélites. Estos se pueden colocar hasta en 5 órbitas en el espacio en función de las necesidades de cada uno. 

- La **órbita LEO** u **órbita baja** es una amplia franja que se sitúa entre los 160 y los 2000 kilómetros de altura. Los objetos situados en esta órbita se mueven a gran velocidad respecto de la superficie terrestre, por lo cubren una órbita completa en pocos minutos o pocas horas. Aquí se sitúan la Estación Espacial Internacional y la mayoría de los satélites meteorológicos de observación y muchos satélites de comunicaciones. 

- La **órbita MEO** u **órbita circular intermedia** se halla entre los 2000 y 36000 kilómetros de altura. Su periodo orbital tiene un promedio de 12 horas y aquí se sitúan los satélites de observación, defensa y posicionamiento, las redes satelitales de GPS y los satélites Glonass rusos o los Galileo europeos. 

- La **órbita geoestacionaria** u **órbita GEO** se sitúa a 35786 kilómetros de la superficie terrestre. Su periodo orbital es de 24 horas y en ella se ubican todos los satélites que proveen internet, televisión, telefonía y otros datos a diferentes regiones del planeta. 

- La **órbita HEO** u **órbita alta** se halla a más de 36000 kilómetros de altura y tiene un periodo orbital de más de 24 horas. Por último, tendríamos ya la **órbita SSO** u **órbita sincrónica solar**.

**Comencemos la exploración.**

**Actividad:** Encuentre el satélite en funcionamiento más antiguo y más reciente.

Determinamos esto usando el método **.iloc()** en el primer y último elemento del marco de datos:

In [ ]:
print(df_sat.iloc[0,0:5].to_string())

In [ ]:
print(df_sat.iloc[-1,0:5].to_string())

La base de datos contiene información sobre satélites, su tipo de órbita y usos principales. Para determinar el satélite más antiguo en órbita geoestacionaria, utilizado con fines **comerciales**, debemos aplicar una _indexación booleana_ utilizando las columnas 'Clase de órbita' y 'Usuarios'

In [ ]:
print(df_sat[(df_sat['Class of Orbit']=='GEO') & (df_sat['Users']=='Commercial')].iloc[0,0:8].to_string())

**Actividad:** Encuentre el satélite más pesado en operaciones

In [ ]:
heaviest_sat = df_sat[['Dry Mass (kg.)']].idxmax()
print(df_sat.loc[heaviest_sat,['Name of Satellite, Alternate Names','Operator/Owner','Class of Orbit','Dry Mass (kg.)']].to_string())

Además de la información sobre la fecha de lanzamiento y el nombre del satélite, la base de datos contiene variables como el país que registró el satélite en la ONU (País de registro de la ONU, país al que pertenece el operador del satélite (País del operador).

_Con esta información podemos explorar qué tan activos son los países en el espacio._

**¿Qué países son más activos en el espacio?** 

**Exploración a nivel de país**: Al observar el _país de registro_ y el _país del operador_, podríamos medir la actividad del sector espacial de un país.

In [ ]:
# Determinar el número de satélites por país de registro y operador, para todos los países.
# La serie devuelta contiene la suma de satélites de todos los países, ordenados por valor.
# Nota: Debemos eliminar las entradas No registradas (NR) de la columna 'País/Org del Registro de la ONU' antes de realizar el conteo.

nbr_sats_per_cnt_reg=df_sat['Country/Org of UN Registry'].value_counts()
nbr_sats_per_cnt_op=df_sat['Country of Operator/Owner'].value_counts()

print('Array sizes: Per country of registration {} per country of operator {}'.format(nbr_sats_per_cnt_reg.shape, nbr_sats_per_cnt_op.shape ))

Las variables **nbr_sats_per_reg** y **nbr_sats_per_op** contienen el número de satélites por país de registro y por país de operador, ordenados por valor ascendente. Como la cantidad de países en cada matriz es bastante grande, la siguiente celda explora los cinco principales de cada variable:

In [ ]:
# Número de satélites según el país de registro, clasificados entre los 5 primeros.
print('Recuento total de satélites (top 5), por país de registro de la ONU')
print('-------------------------------------------------------------')
print(df_sat[df_sat['Country/Org of UN Registry'].str.contains("NR")==0]['Country/Org of UN Registry'].value_counts()[:5].to_string())

In [ ]:
# Número de satélites según el país del operador/propietario, clasificados entre los 5 primeros.
print('Recuento total de satélites (top 5), por país del operador')
print('------------------------------------------------------')
print(df_sat['Country of Operator/Owner'].value_counts()[:5].to_string())

**NOTA: Puede ser mejor visualizar esta información, en lugar de presentarla como texto.**

In [ ]:
# un gráfico de barras para los cinco países principales,
fig, ax = plt.subplots(nrows=1, ncols=2)
fig.tight_layout(pad=0.0)
fig.set_size_inches(14,6)
ax1=df_sat[df_sat['Country/Org of UN Registry'].str.contains("NR")==0]['Country/Org of UN Registry'].value_counts()[:5].plot(kind='barh',ax=ax[0])
ax2=nbr_sats_per_cnt_op[0:5].plot(kind='barh')
plt.subplots_adjust(right=1.5,wspace=0.5)

# establecer una leyenda, título
ax1.set_xlabel('Número de satélites en la base de datos', fontsize=18)
ax1.set_title('Número total de satélites, por país del Registro de la ONU', fontsize=20)
ax2.set_xlabel('Número de satélites en la base de datos', fontsize=18)
ax2.set_title('Número total de satélites, por país del Operador', fontsize=20)

# anotar
set_bar_label(ax1)
set_bar_label(ax2)

Los resultados probablemente no sean una sorpresa: los países que han registrado la mayoría de los satélites y que operan la mayoría de los satélites son **EE. UU.**, **China** y **Rusia**.

**¿Desde dónde se lanzan los satélites?**

In [ ]:
axTen = df_sat['Launch Site'].value_counts()[:10].plot(kind='barh', color='purple',width=0.8, figsize=(8,8))
set_bar_label(axTen, orient='h')
_=axTen.set_title('Número total de lanzamientos, por sitio de lanzamiento (10 principales)', fontsize=20)

In [ ]:
nbr_sats_per_launch_site=df_sat['Launch Site'].value_counts()
print('Lista de los 10 principales sitios de lanzamiento, ordenados por número de satélites lanzados')
print('--------------------------------------------------------------------')
print(nbr_sats_per_launch_site[0:10].to_string())

**¿Y qué vehículos se utilizan para lanzarlos?**

También podemos repetir el principio para los vehículos de lanzamiento, usando los datos en la columna "Vehículo de lanzamiento". Tenga en cuenta que

- Existe lanzamiento de varios vehículos desde el mismo sitio.

- Los vehículos de lanzamiento pueden lanzarse desde diferentes sitios según la misión.

In [ ]:
nbr_sats_per_launch_vehicle=df_sat['Launch Vehicle'].value_counts()
print('Lista de lanzadores por número de satélites lanzados en total, 10 principales')
print('---------------------------------------------------------------')
print(nbr_sats_per_launch_vehicle[0:10].to_string())

In [ ]:
BySite_group = df_sat.groupby(['Launch Site','Launch Vehicle']) 
nbr_Starlink_launched_from_cc = BySite_group.get_group(('Cape Canaveral','Falcon 9'))['Name of Satellite, Alternate Names'].str.contains('tarlink').sum()
nbr_NonStarlink_launched_from_cc = BySite_group.get_group(('Cape Canaveral','Falcon 9'))['Name of Satellite, Alternate Names'].str.contains('^((?!tarlink).)*$').sum()

print(' - Número de satélites Starlink lanzados con Falcon 9 desde Cabo Cañaveral : ', nbr_Starlink_launched_from_cc )
print(' - Número de satélites que no son Starlink lanzados con Falcon 9 desde Cabo Cañaveral : ', nbr_NonStarlink_launched_from_cc )
print(' - Total lanzado desde Cabo Cañaveral usando Falcon 9: ', nbr_Starlink_launched_from_cc+nbr_NonStarlink_launched_from_cc)

Ahora, una mirada más cercana a los primeros 15 satélites lanzados con Falcon 9.

In [ ]:
# Para modificar el código para ver, digamos, solo los primeros 10 lanzamientos, 
# agregue [0:11] después de los nombres de las columnas en la función de impresión ().
ByLaunchSite = df_sat.groupby(['Launch Vehicle'])
display(ByLaunchSite.get_group('Falcon 9')[['Launch Site','Name of Satellite, Alternate Names']][0:16])

**¿Qué órbitas se utilizan con más frecuencia?**

La base de datos incluye información sobre la clase de órbita. Pero, **¿qué es una órbita?** 

- Una órbita es el camino que toma un objeto cuando se mueve alrededor de otro, debido a la gravedad.

Construyamos un histograma para explorar la cantidad de satélites operativos por _clase de órbita_. 

In [ ]:
fig_class, ax_class = plt.subplots(nrows=1, ncols=1)
fig_class.set_size_inches(10,10)
ax_class.tick_params(axis='x', labelsize=14)
ax_class.tick_params(axis='y', labelsize=14)

axClassBar = sns.countplot(df_sat['Class of Orbit'])
axClassBar.set_xlabel('Clase de órbita',fontsize=18)
_=axClassBar.set_title('Número total de satélites, por clase de órbita', fontsize=20)
set_bar_label(axClassBar,minval=1, orient='v')

La base de datos también contiene información sobre para qué se utilizan los satélites. Podemos responder preguntas como,

**¿Cuántos satélites de comunicación con fines comerciales están en operación, agrupados por clase de órbita?**

In [ ]:
cross_tab = pd.crosstab( df_sat['Users'], df_sat['Class of Orbit'],margins=True)
display(cross_tab)

Dado que el conjunto de datos tiene usos mixtos, necesitaríamos algunas líneas de código más para sumar todas las categorías que contienen un uso comercial, para cada tipo de órbita. A continuación se muestra una forma de hacerlo.

In [ ]:
gb=df_sat.groupby(['Class of Orbit','Users'])

# calcular el total por clase de órbita, utilizando métodos de cálculo directo. 
# Tenga en cuenta que esto también se puede lograr a través de filtros e indexación lógica.
nbr_GEO_comm_sats = gb.get_group(('GEO','Commercial'))['Name of Satellite, Alternate Names'].count() + \
              gb.get_group(('GEO','Commercial/Military'))['Name of Satellite, Alternate Names'].count() + \
              gb.get_group(('GEO','Commercial/Government'))['Name of Satellite, Alternate Names'].count()

nbr_MEO_comm_sats = gb.get_group(('MEO','Commercial'))['Name of Satellite, Alternate Names'].count() #+ \
             # gb.get_group(('MEO','Military/Commercial'))['Name of Satellite, Alternate Names'].count()

nbr_LEO_comm_sats = gb.get_group(('LEO','Commercial'))['Name of Satellite, Alternate Names'].count() + \
              gb.get_group(('LEO','Commercial/Civil'))['Name of Satellite, Alternate Names'].count() + \
              gb.get_group(('LEO','Government/Commercial'))['Name of Satellite, Alternate Names'].count() +\
              gb.get_group(('LEO','Military/Commercial'))['Name of Satellite, Alternate Names'].count()

print('Totales por clase de órbita')
print('-------------------------------------------------')
print('Número de satélites GEO para uso comercial: ', nbr_GEO_comm_sats)
print('Número de satélites MEO para uso comercial: ', nbr_MEO_comm_sats)
print('Número de satélites LEO para uso comercial: ', nbr_LEO_comm_sats)

**¿Cuántos satélites se utilizan para servicios de telecomunicaciones, por tipo de órbita?**

Podemos subdividir aún más los datos por órbita y usuario y centrarnos en un propósito específico, el de los _servicios de comunicación comercial_.

In [ ]:
# Para que este conteo funcione, es importante recordar que hay ciertas entradas en la base de datos de "uso mixto", 
# es decir, un satélite con cargas útiles con propósitos duales, como comercial/militar o gubernamental/militar.
# Esto es lo que se utilizó una suma directa de cada categoría en las celdas anteriores.
# Ahora se intentará un enfoque de indexación lógica usando el método str.contains() 
# y buscando las palabras clave Comercial y Comunicación dentro de las columnas Usuarios y Propósito.

idx_isLEO_isCOM_isCOMMS = (df_sat['Class of Orbit']=='LEO') & (df_sat['Users'].str.contains('commercial',case=False) ) & (df_sat['Purpose'].str.contains('communication',case=False))
idx_isMEO_isCOM_isCOMMS = (df_sat['Class of Orbit']=='MEO') & (df_sat['Users'].str.contains('commercial',case=False) ) & (df_sat['Purpose'].str.contains('communication',case=False))
idx_isGEO_isCOM_isCOMMS = (df_sat['Class of Orbit']=='GEO') & (df_sat['Users'].str.contains('commercial',case=False) ) & (df_sat['Purpose'].str.contains('communication',case=False))

# el número de entradas es la suma de la serie de indexación booleana.
nbr_LEO_com_comms_sats =idx_isLEO_isCOM_isCOMMS.sum()
nbr_MEO_com_comms_sats =idx_isMEO_isCOM_isCOMMS.sum()
nbr_GEO_com_comms_sats =idx_isGEO_isCOM_isCOMMS.sum()

print('Número total de satélites comerciales de comunicaciones, por clase de órbita')
print('------------------------------------------------------------------------')
print('Número de satélites LEO para uso de comunicaciones comerciales: ', nbr_LEO_com_comms_sats)
print('Número de satélites MEO para uso de comunicaciones comerciales: ', nbr_MEO_com_comms_sats)
print('Número de satélites GEO para uso de comunicaciones comerciales: ', nbr_GEO_com_comms_sats)

# un gráfico de barras
fbar,barax=plt.subplots()
fbar.set_size_inches(7,10)
barax.bar( ['LEO', 'MEO', 'GEO'], [nbr_LEO_com_comms_sats, nbr_MEO_com_comms_sats, nbr_GEO_com_comms_sats], color=['r', 'g','b'])

# anotar barras,
set_bar_label(barax,minval=1, orient='v')
barax.set_ylabel('Número de satélites comerciales con fines de comunicaciones', fontsize=14)
plt.show()

Por lo tanto, hay casi tres veces más satélites LEO con fines comerciales que satélites GEO. 

**¿Significa esto que los sistemas LEO son más exitosos que los sistemas GEO?** 

No necesariamente. Los satélites en la órbita LEO se mueven muy rápido en relación con un punto fijo en la tierra. Los satélites GEO, en cambio, aparecen fijos en un punto de la tierra. Entonces, para una cobertura o servicio continuo, los sistemas desplegados en la órbita LEO necesitan más de un satélite, necesitan una constelación.  

Algunas constelaciones bien conocidas de servicios de comunicaciones incluyen **Iridium, Orbcomm, Globalstar** y los relativamente nuevos **OneWeb** y **SpaceX Systems**. Los satélites LEO tienden a ser más pequeños (y livianos) que sus contrapartes GEO, ya que la naturaleza de la carga útil es diferente. 

Exploremos la velocidad orbital media de los satélites LEO y la relación entre las capacidades de masa y potencia de los satélites para ver si esto es cierto.

**Explorando los operadores de sistemas satelitales**

Ahora cambiemos ligeramente de tema y echemos un vistazo a la _cantidad de operadores de satélites_ que vuelan satélites con fines comerciales.

In [ ]:
nbr_sats_per_op=df_sat['Operator/Owner'].value_counts()

# un gráfico de barras para los diez principales operadores, por número de satélites,
axOp=df_sat['Operator/Owner'].value_counts()[:10].plot(kind='barh', width =0.8, figsize=(8,8))

# anotar barras,
set_bar_label(axOp)

# establecer una leyenda, título
axOp.set_xlabel('Número de satélites en la base de datos')
_= axOp.set_title('Número total de satélites, por operador', Fontsize=20)

Como se puede observar, OneWeb y SpaceX lideran la lista por número de satélites. Cuatro de los operadores con más satélites son relativamente nuevos, y dos de los cinco principales están en el dominio de la observación de la Tierra.

Cambiar el enfoque a operadores GEO, con énfasis en SATCOM comerciales,

In [ ]:
# Una mirada más cercana a los operadores GSO, con un enfoque comercial en el dominio de las telecomunicaciones, arroja los siguientes tamaños de flota 
# Primero creamos una lista con los nombres de los operadores,
GSO_comm_ops = np.unique(df_sat['Operator/Owner'][(df_sat['Users'] == 'Commercial') & (df_sat['Purpose'] == 'Communications')& (df_sat['Class of Orbit'] == 'GEO')])

# la longitud de la lista es 74, lo que significa que hay 74 operadores que cumplen las tres condiciones establecidas en nuestra consulta anterior.
print('Hay {} operadores GEO satcomm en la lista'.format(len(GSO_comm_ops)))
print('----------------------------------------------')

# alternativamente, podemos usar un enfoque más complejo pero compacto, 
# como hicimos en las figuras anteriores, usando Value_counts()
ax4 = df_sat['Operator/Owner'][(df_sat['Users'] == 'Commercial') & (df_sat['Purpose'] == 'Communications') \
                              & (df_sat['Class of Orbit'] == 'GEO')].value_counts()[0:10].plot(kind='barh',figsize=(8,8))
_=set_bar_label(ax4)

Un problema con el análisis anterior es que el nombre de un operador de satélite aparece varias veces, en forma diferente. Por ejemplo, en los casos en que comparte la plataforma con otro operador. Por ejemplo, un satélite podría asignarse a un operador como

- Telesat Canadá,
- Telesat Canada Ltd. (BCE, Inc.)/APT Satellite Holdings Ltd.

que se registrarán como operadores separados. Además, las entidades fusionadas aparecen como separadas (ver Panamsat/Intelsat o SES/O3B).

Hagamos una búsqueda más específica, para lo que se conoce como los "Cuatro Grandes" operadores FSS: **Eutelsat, Intelsat, SES, Telesat**, buscando cualquier satélite que tenga esas compañías listadas como operadores en cualquier combinación en la cadena, y, en el caso de SES, añadiendo la constelación O3B en órbita MEO.

In [ ]:
IS_count=np.sum(df_sat['Operator/Owner'].str.contains('intelsat',case=False) & df_sat['Users'].str.contains('Commercial'))
EUT_count=np.sum(df_sat['Operator/Owner'].str.contains('eutelsat',case=False) & df_sat['Users'].str.contains('Commercial'))
TEL_count=np.sum(df_sat['Operator/Owner'].str.contains('telesat',case=False) & df_sat['Users'].str.contains('Commercial'))
SES_count=np.sum(df_sat['Operator/Owner'].str.contains('ses',case=False) & df_sat['Users'].str.contains('Commercial')) + \
          np.sum(df_sat['Operator/Owner'].str.contains('o3b',case=False) & df_sat['Users'].str.contains('Commercial'))

print('UCS contiene : ')
print('---------------------------')
print(' {} satélites Eutelsat'.format(EUT_count))
print(' {} satélites Intelsat'.format(IS_count))
print(' {} satélites SES/O3B'.format(SES_count))
print(' {} satélites Telesat'.format(TEL_count))

**¿Para qué se utilizan los satélites de comunicaciones GEO?**

Podemos revisar los usos de los satélites de órbita geoestacionaria estudiando las categorías en la columna "Usuarios" del marco de datos

In [ ]:
# extraiga todos los puntos de datos para la órbita GEO, mire la columna de longitud
geo_flt_id = (df_sat['Class of Orbit'] == 'GEO')
GEO_df = df_sat[geo_flt_id]
GEO_byUser=GEO_df.groupby(['Users'])['Purpose'].count()
f=GEO_byUser.plot(kind='bar',figsize=(10,8), fontsize=16, width=1, position=0.5)
f.set_title('Usuarios registrados de satélites comerciales geoestacionarios', fontsize=18)
_=set_bar_label(f,1,'v')

print(GEO_byUser)

**¿Dónde se encuentran los satélites de comunicación GEO?**

**Actividad**: Filtrar los datos según la órbita GEO y el uso de comunicaciones comerciales, y hacer un histograma de los puntos de datos válidos en el conjunto resultante, nos da una idea de la distribución de los satélites a lo largo de la órbita GEO.

In [ ]:
# extraiga todos los puntos de datos para la órbita GEO, mire la columna de longitud
flt_id = (df_sat['Class of Orbit'] == 'GEO') & (df_sat['Users'].str.contains('commercial', case = False) & (df_sat['Purpose'].str.contains('communications', case = False) ) )
GEO_long = df_sat[ flt_id ]['Longitude of GEO (degrees)'].astype(float)

# longitudes correctas para caer en el rango (-180,180)
GEO_long[ GEO_long > 180] = GEO_long[ GEO_long > 180] - 360

# hacer un histograma de los puntos de datos de longitud,
# definición de intervalos de histograma y ubicaciones de xticks.
bin_edges = np.arange(-180,185,5)-2.5 # cada cinco grados.
x_axis_ticks = np.arange(-180,190,10)

# crear figura lienzo y ejes
fig, ax5 = plt.subplots(figsize=[15,8])
fig.tight_layout(pad=0.0)

# gráfico.
_ = sns.distplot(GEO_long,bins=bin_edges,kde=False,axlabel='Orbital location (deg E.)',ax=ax5, hist_kws ={'ec':'k'}) 
_ = plt.xticks(x_axis_ticks,rotation=55)
_ = plt.xlabel('Ubicación orbital (deg E.)')
_ = plt.ylabel('Número de satélites en el contenedor de ubicación orbital')

set_bar_label(ax5, minval=10, orient='v')

Con las definiciones de intervalo anteriores, hay tres "contenedores" con 10 o más satélites. 

**¿Por qué algunas ubicaciones orbitales parecen más atractivas que otras?** 

- Hay varias razones. Desde la perspectiva de las comunicaciones, tiene que ver con los mercados a los que llega desde cada ubicación orbital.

- Por ejemplo, un contribuyente a la cantidad de satélites alrededor de -100 grados Este y -60 grados Oriente es el hecho de que alrededor de esos lugares hay bastantes satélites que brindan servicios de video a las cabeceras de cable (servicios de distribución de cable) y a los usuarios en el hogar (DTH).

- Hay puntos de acceso similares en Europa, por ejemplo, el conjunto 16E y 19.2E que admiten DTH. Eso contribuye a que la cantidad de satélites alrededor de 20E y 25E esté cerca de 8.

- Alrededor de 30 grados hay bastantes satélites, debido a que el tamaño del contenedor es grande. Ese intervalo está capturando 26E, 28.2E, 30E, 31E, 33E, que son responsables de una gran cantidad de servicios de video y datos hacia Europa y Medio Oriente, con satélites de **Arabsat, Eutelsat, Intelsat** y **SES** en esos espacios.

**Sistemas de navegación**

Teniendo en cuenta las noticias sobre el sistema BEIDOU de China, veamos el tamaño de tres constelaciones de navegación: el **sistema GPS** de EE. UU. (DoD/US Air Force GPS system), el **sistema GALILEO** de Europa y el **BEIDOU** de China.

Para contar cuántos satélites hay en cada constelación, se hará una búsqueda de cadenas en el nombre y se contarán las coincidencias. Esto capturará cualquier satélite, independientemente de su órbita (el sistema Beidou tiene satélites en órbita MEO y GEO):

In [ ]:
DOD_GPS = df_sat['Name of Satellite, Alternate Names'].str.contains('navstar gps',case=False).sum()
EU_GALILEO = df_sat['Name of Satellite, Alternate Names'].str.contains('galileo',case=False).sum()
CH_BEIDOU = df_sat['Name of Satellite, Alternate Names'].str.contains('beidou',case=False).sum()
print('Recuento de satélites operativos (ENE-2022) en tres constelaciones de navegación:')
print('-----------------------------------------------------------------------------')
print('US DoD GPS    : {} satélites '.format(DOD_GPS))
print('EU GALILEO    : {} satélites '.format(EU_GALILEO))
print('CHINA BEIDOU  : {} satélites '.format(CH_BEIDOU))

# EJERCICIO-01: Explorando números por contratista

Finalmente, dada la información en la columna Contractor ("Contratista"), también es posible evaluar el número de naves espaciales construidas por cada fabricante. Tenga en cuenta que, al igual que con el nombre del operador, es posible que las naves espaciales construidas por un fabricante en la actualidad deban agregarse a partir de varias entradas.

**Sin fusionar fabricantes, presente un gráfico de barras para los 15 mejores resultados por número de satélites**.

# EJERCICIO-02: ¿Qué tan rápido se mueven los satélites? 

**Calcular la velocidad orbital media de los satélites**

La velocidad orbital media de un satélite que orbita alrededor de la Tierra es una función del período orbital y la longitud del semieje mayor de la elipse de la órbita. Podemos aproximar la velocidad orbital media usando

$$\frac{2\pi a}{T}(1-0.25e^{2})$$

donde

- $a$ : longitud del semieje mayor en kilómetros,
- $T$ : período de órbita en segundos
- $e$ : excentricidad de la órbita
    
La velocidad orbital media no está en la base de datos. Podemos agregar una columna al marco de datos con esos valores

In [ ]:
R_earth=6378.165  # en kilómetros, radio medio de la tierra.
df_sat['Mean Orbital Speed (km.sec)'] = ((2*np.pi*( (df_sat['Perigee (km)'] + df_sat['Apogee (km)'] +2*R_earth)/2)) / (df_sat['Period (minutes)']*60))*(1-0.25*df_sat['Eccentricity']**2)
display( df_sat[['Name of Satellite, Alternate Names','Class of Orbit','Mean Orbital Speed (km.sec)']].head(6))

# criterios booleanos para el filtrado
criteria_LEO = df_sat['Class of Orbit'] == 'LEO'
criteria_MEO = df_sat['Class of Orbit'] == 'MEO'
criteria_GEO = df_sat['Class of Orbit'] == 'GEO'

# velocidad orbital media para cualquier entrada distinta de cero o nula para satélites LEO
print(' ------------------Velocidad orbital media: estadística descriptiva LEO--------------------------------- ')
Mean_Orbital_speed_LEO = df_sat[criteria_LEO]['Mean Orbital Speed (km.sec)'].apply(pd.to_numeric,errors='coerce').dropna().mean()
print(df_sat[criteria_LEO][ 'Mean Orbital Speed (km.sec)'].dropna().describe().to_string())
print( 'Velocidad orbital media para satélites LEO con conjunto completo de parámetros en UCS: {0:6.2f} km/sec.'.format ( Mean_Orbital_speed_LEO) )

# para cualquier entrada distinta de cero o nula para satélites MEO

# para cualquier entrada distinta de cero o nula para satélites GEO


# EJERCICIO-03: Explorando las relaciones básicas entre los atributos de los satélites 

**La relación _Masa-Potencia_**. 
    
La base de datos contiene muestras de las capacidades de _masa_ y _potencia_ de las naves espaciales. Históricamente, se ha basado en la relación entre la masa (masa seca, masa de la carga útil, masa propulsora) y la potencia para establecer reglas de primer orden para estimar la masa de la nave espacial, conociendo una envolvente de potencia. Explore entonces la relación entre la _masa seca_ de la nave espacial y la _capacidad de potencia_.

In [ ]:
criteria_LEO = df_sat['Class of Orbit'] == 'LEO'
print('Masa seca promedio para un satélite LEO: {0:6.2f} kg.'.format(df_sat[criteria_LEO]['Dry Mass (kg.)'].dropna().median()))
print('Capacidad promedio de generación de energía para un satélite LEO: {0:6.2f} Watt.'.format( df_sat[criteria_LEO]['Power (watts)'].dropna().median()))
print('----------------------------------------------------------------------')

# Extraiga los mismos datos para los satélites MEO

# Extraiga los mismos datos para los satélites GEO


In [ ]:
df_sat[criteria_LEO][['Name of Satellite, Alternate Names','Dry Mass (kg.)']][df_sat[criteria_LEO]['Dry Mass (kg.)']>2000]

# EJERCICIO-04 

Muestre un **gráfico de dispersión** con las variables _Masa seca, Potencia_ y _Clase de órbita_ (GEO, MEO, LEO, Elliptical).   